## 맛있는 절약 : 평가용 dataset 생성
1. find_keyIngredients_tasty : (사전 생성) 레시피의 핵심 맛과 핵심 재료를 판단
2. generate_food_group_radio : (사전 생성) 레시피의 식품군을 판별
3. fridge_recipe_transform : (기능) 냉장고 재료 대체 
4. simple_recipe_transform : (기능) 레시피 간소화 
5. balance_nutrition : (기능) 영양 맞춤형 레시피 

## 일반적인 평가용 dataset 생성 case
- 검색한 것이 질문과 관련이 있는가?
- 답변이 질문과 관련이 있는가?
- 답변을 검색한 문서 안에서 답변한 것인가?(할루시네이션 check)

## 한계
검색한 결과에 대한 정확한 기준 데이터(Ground truth) 구축이 사실상 어려움
따라서, 검색한 결과에 대한 Ground truth가 존재한다면 모두 데이터셋으로 활용하고, 아니라면 질문과 답변만으로 데이터셋을 구축하여 활용함
⇒ 우리는 레시피 생성시에 RAG가 없기 때문에 질문과 답변만으로 데이터셋을 구축해서 활용함

In [4]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [5]:
# from langchain_teddynote import logging

# # 프로젝트 이름을 입력합니다.
# logging.langsmith("맛있는 절약-Evaluations")

LangSmith 추적을 시작합니다.
[프로젝트명]
맛있는 절약-Evaluations


## 레시피 dataset
spicy_level, type_key, method_key, servings를 기준으로 각각 중복되지 않은 행만 추출 

In [18]:
import pandas as pd
df_recipeinfo = pd.read_csv("data/diverse_recipes_dataset.csv")
df_recipeinfo.describe()

,_id,title,type_key,method_key,servings,cooking_time,difficulty,ingredients,cooking_steps,tips,recipe_type,spicy_level
count,37,37,37,37,37,33,35,37,37,37,37,37
unique,37,37,12,13,6,7,2,37,37,22,37,6
top,673c36d38fad4e3c931171ba,"토마토 마리네이드&냉파스타, 새콤달콤 시원한 별미",반찬,볶음,2인분,30분 이내,초급,"['방울토마토(1.5kg)', '양파(1개)', '파슬리가루(1스푼)', '올리브오...","['양파는 잘게 다져 준비해주세요양파(소) 1개', '방울토마토는 끓는물에 약 2분...",[],"['채식', '저칼로리', '새콤달콤한맛', '양식', '가벼운']",매운맛없음
freq,1,1,12,8,12,11,34,1,1,16,1,25


In [24]:
df_userinfo = pd.read_csv('data/persona_csv.csv')
df_userinfo.describe()

,persona,user_allergy_ingredients,user_dislike_ingredients,user_spicy_level,user_cooking_level,user_owned_ingredients,user_basic_seasoning,must_use_ingredients
count,10,10,10,10,10,10,10,10
unique,10,4,10,5,3,10,10,8
top,유미,[],[],3단계,초급,"['닭가슴살', '두부', '브로콜리']","['소금', '후추', '올리브유']",['닭가슴살']
freq,1,7,1,3,5,1,1,2


In [25]:
import pandas as pd

# Recipe와 User 정보를 균등하게 결합하여 JSON 형식으로 변환
def create_combined_dataset(recipeinfo_df, userinfo_df):
    combined_data = []
    len_userinfo = len(userinfo_df)  # UserInfo 데이터 수
    
    for idx, recipe_row in recipeinfo_df.iterrows():
        user_idx = idx % len_userinfo  # user_info를 균등하게 분배하기 위한 인덱스
        user_row = userinfo_df.iloc[user_idx]
        
        combined_entry = {
            "user_info": {
                "user_allergy_ingredients": eval(user_row["user_allergy_ingredients"]),
                "user_dislike_ingredients": eval(user_row["user_dislike_ingredients"]),
                "user_spicy_level": user_row["user_spicy_level"],
                "user_cooking_level": user_row["user_cooking_level"],
                "user_owned_ingredients": eval(user_row["user_owned_ingredients"]),
                "user_basic_seasoning": eval(user_row["user_basic_seasoning"]),
                "must_use_ingredients": eval(user_row["must_use_ingredients"]),
            },
            "recipe_info": {
                "_id": recipe_row["_id"],
                "title": recipe_row["title"],
                "type_key": recipe_row["type_key"],
                "method_key": recipe_row["method_key"],
                "servings": recipe_row["servings"],
                "cooking_time": recipe_row["cooking_time"] if not pd.isna(recipe_row["cooking_time"]) else "",
                "difficulty": recipe_row["difficulty"] if not pd.isna(recipe_row["difficulty"]) else "",
                "ingredients": eval(recipe_row["ingredients"]),
                "cooking_steps": eval(recipe_row["cooking_steps"]),
                "tips": eval(recipe_row["tips"]),
            },
        }
        combined_data.append(combined_entry)
    
    return combined_data

# 데이터셋 생성
combined_dataset = create_combined_dataset(df_recipeinfo, df_userinfo)

# JSON 데이터셋 저장 (옵션)
import json
with open('data/combined_dataset.json', 'w', encoding='utf-8') as f:
    json.dump(combined_dataset, f, ensure_ascii=False, indent=4)

print("데이터셋 생성 완료. 첫 번째 데이터:")
print(combined_dataset[0])
print("갯수:", len(combined_dataset))


데이터셋 생성 완료. 첫 번째 데이터:
{'user_info': {'user_allergy_ingredients': [], 'user_dislike_ingredients': [], 'user_spicy_level': '3단계', 'user_cooking_level': '중급', 'user_owned_ingredients': ['닭가슴살', '두부', '브로콜리'], 'user_basic_seasoning': ['소금', '후추', '올리브유'], 'must_use_ingredients': ['닭가슴살']}, 'recipe_info': {'_id': '673c36d38fad4e3c931171ba', 'title': '토마토 마리네이드&냉파스타, 새콤달콤 시원한 별미', 'type_key': '샐러드', 'method_key': '절임', 'servings': '1인분', 'cooking_time': '60분 이내', 'difficulty': '초급', 'ingredients': ['방울토마토(1.5kg)', '양파(1개)', '파슬리가루(1스푼)', '올리브오일(10스푼)', '발사믹식초(5스푼)', '레몬즙(2스푼)', '꿀(1.5스푼)', '소금(약간)', '통후추(약간)', '토마토 마리네이드(1컵)', '파스타면(1컵)', '소금(1스푼)', '파마산치즈(1스푼)'], 'cooking_steps': ['양파는 잘게 다져 준비해주세요양파(소) 1개', '방울토마토는 끓는물에 약 2분간 데쳐주세요. 칼집을 내면 좀더 편하긴 한데 안해도 상관없어요방울토마토 1팩(약 1.5키로)중간불', '데친 후에는 물을 버리고 찬물에 담궈 식혀주고 토마토 껍질을 벗겨줍니다.칼집을 내지않아도 이렇게 껍질이 터져서 벗기기 쉽고 터져있지 않더라도 껍질 분리가 잘 된답니다^^', '껍질을 벗긴 토마토와 다진 양파, 파슬리가루를 넣어주세요step1의 다진양파, 파슬리가루 약 1스푼', '올리브오일, 레몬즙, 발사믹식초, 소금, 후추, 꿀을 요리재료에 적힌 분량대로 넣어준 뒤 잘 섞어

In [26]:
import pandas as pd

# JSON 파일 경로
file_path = 'data/combined_dataset.json'

# JSON 파일을 DataFrame으로 불러오기
df = pd.read_json(file_path)

for i in range(len(df)):
    print(df.user_info[i])


{'user_allergy_ingredients': [], 'user_dislike_ingredients': [], 'user_spicy_level': '3단계', 'user_cooking_level': '중급', 'user_owned_ingredients': ['닭가슴살', '두부', '브로콜리'], 'user_basic_seasoning': ['소금', '후추', '올리브유'], 'must_use_ingredients': ['닭가슴살']}
{'user_allergy_ingredients': [], 'user_dislike_ingredients': ['토마토', '오이'], 'user_spicy_level': '2단계', 'user_cooking_level': '초급', 'user_owned_ingredients': ['계란', '감자', '스팸'], 'user_basic_seasoning': ['간장', '설탕', '후추'], 'must_use_ingredients': ['계란']}
{'user_allergy_ingredients': [], 'user_dislike_ingredients': ['고수'], 'user_spicy_level': '4단계', 'user_cooking_level': '고급', 'user_owned_ingredients': ['버섯', '파스타면', '레몬'], 'user_basic_seasoning': ['올리브유', '소금', '파프리카 가루'], 'must_use_ingredients': ['버섯']}
{'user_allergy_ingredients': [], 'user_dislike_ingredients': ['치즈'], 'user_spicy_level': '3단계', 'user_cooking_level': '초급', 'user_owned_ingredients': ['양파', '라면', '김치'], 'user_basic_seasoning': ['소금', '고춧가루', '간장'], 'must_use_ingredients': ['

## 작은 datatset 생성

In [27]:
df[:1]

,user_info,recipe_info
0,"{'user_allergy_ingredients': [], 'user_dislike...","{'_id': '673c36d38fad4e3c931171ba', 'title': '..."


In [28]:
import json
with open('data/small_combined_dataset.json', 'w', encoding='utf-8') as f:
    json.dump(combined_dataset[:2], f, ensure_ascii=False, indent=4)

### dataset에 find_keyIngredients_tasty 결과 추가

In [97]:
import sys
sys.path.append('../')  # 상위 디렉토리의 src 폴더를 경로에 추가

from src.recipe_change_origin import langfuse_tracking, get_system_prompt, choose_feature, ChangeRecipe, RecipeChangeBalanceNutrition 
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser, StrOutputParser
from src.logger import logger_eval
import pandas as pd
import asyncio

llm = ChatOpenAI(
        model="gpt-4o",
        temperature=0.0,
        max_tokens=1000,
        timeout=40
    )
logger_eval.info("LLM 초기화 완료.")

# async def generate_test_dataset(prompt, df, new_column):
#     prompt = get_system_prompt(prompt)
#     llm_chain = prompt | llm | StrOutputParser()

#     logger_eval.info("LLM 레시피 생성 중...")
#     for i in range(len(df)):
#         langfuse_handler = langfuse_tracking()
#         result = await llm_chain.ainvoke(input={"user_info":df.user_info[i], "recipe_info":df.recipe_info[i]}, config={"callbacks": [langfuse_handler]})
#         df.at[i, new_column] = result
#     logger_eval.info("LLM 레시피 생성 완료")
#     return df


In [98]:
async def generate_test_dataset(prompt, df, new_column, batch_size=5):
    prompt = get_system_prompt(prompt)
    llm_chain = prompt | llm | StrOutputParser()

    logger_eval.info("LLM 레시피 생성 중...")
    for start in range(0, len(df), batch_size):
        end = start + batch_size
        langfuse_handler = langfuse_tracking()
        tasks = [
            llm_chain.ainvoke(
                input={"user_info": df.user_info[i], "recipe_info": df.recipe_info[i]},
                config={"callbacks": [langfuse_handler]},
            )
            for i in range(start, min(end, len(df)))
        ]
        results = await asyncio.gather(*tasks, return_exceptions=True)
        for i, result in enumerate(results):
            if isinstance(result, Exception):
                logger_eval.error(f"Error at index {start + i}: {result}")
            else:
                df.at[start + i, new_column] = result
    logger_eval.info("LLM 레시피 생성 완료")
    return df


In [38]:
# user_info, recipe_info가 있는 dataset -> 결과는 langfuse 2024.12.18 오전 9:9~9:10분꺼부터 보기
df = pd.read_json('data/combined_dataset.json')

# 비동기 함수 실행
df_tasty = await generate_test_dataset("find_keyIngredients_tasty", df, "recipe_keyIngredients_tasty")
df_tasty.describe()

,user_info,recipe_info,recipe_keyIngredients_tasty
count,37,37,37
unique,10,37,37
top,"{'user_allergy_ingredients': [], 'user_dislike...","{'_id': '673c36d38fad4e3c931171ba', 'title': '...",### 레시피 맛 설명\n이 레시피는 새콤달콤하고 시원한 맛이 특징입니다. 발사믹 ...
freq,4,1,1


In [75]:
df_tasty.iloc[:,:-1].to_csv("tasty_csv", index=False)

In [100]:
df = pd.read_csv("tasty_csv")
df_cp = df.copy().reset_index(drop=True)
final_df = await generate_test_dataset("generate_food_group_ratio", df_cp, "original_recipe_food_group_composition")

In [103]:
final_df.describe()

,user_info,recipe_info,recipe_keyIngredients_tasty,original_recipe_food_group_composition
count,37,37,37,22
unique,10,37,37,22
top,"{'user_allergy_ingredients': [], 'user_dislike_ingredients': [], 'user_spicy_level': '3단계', 'user_cooking_level': '중급', 'user_owned_ingredients': ['닭가슴살', '두부', '브로콜리'], 'user_basic_seasoning': ['소금', '후추', '올리브유'], 'must_use_ingredients': ['닭가슴살']}","{'_id': '673c36d38fad4e3c931171ba', 'title': '토마토 마리네이드&냉파스타, 새콤달콤 시원한 별미', 'type_key': '샐러드', 'method_key': '절임', 'servings': '1인분', 'cooking_time': '60분 이내', 'difficulty': '초급', 'ingredients': ['방울토마토(1.5kg)', '양파(1개)', '파슬리가루(1스푼)', '올리브오일(10스푼)', '발사믹식초(5스푼)', '레몬즙(2스푼)', '꿀(1.5스푼)', '소금(약간)', '통후추(약간)', '토마토 마리네이드(1컵)', '파스타면(1컵)', '소금(1스푼)', '파마산치즈(1스푼)'], 'cooking_steps': ['양파는 잘게 다져 준비해주세요양파(소) 1개', '방울토마토는 끓는물에 약 2분간 데쳐주세요. 칼집을 내면 좀더 편하긴 한데 안해도 상관없어요방울토마토 1팩(약 1.5키로)중간불', '데친 후에는 물을 버리고 찬물에 담궈 식혀주고 토마토 껍질을 벗겨줍니다.칼집을 내지않아도 이렇게 껍질이 터져서 벗기기 쉽고 터져있지 않더라도 껍질 분리가 잘 된답니다^^', '껍질을 벗긴 토마토와 다진 양파, 파슬리가루를 넣어주세요step1의 다진양파, 파슬리가루 약 1스푼', '올리브오일, 레몬즙, 발사믹식초, 소금, 후추, 꿀을 요리재료에 적힌 분량대로 넣어준 뒤 잘 섞어주세요올리브오일 10스푼, 발사믹식초 5스푼, 레몬즙 2스푼 꿀 1.5스푼, 소금 약간, 통후추 약간', '완성된 토마토 마리네이드는 소독한 유리병에 담아 보관하고 먹을만큼 꺼내먹으면 됩니다^^', '소금 약 1스푼을 넣고 파스타를 심지가 씹히지않게 푹 삶아주세요소금 1스푼, 파스타(펜네) 1컵, 물 약500ml강불파스타면이나 푸실리, 마카로니 등 사용하셔도 됩니다.', '다 삶은 파스타면은 물기를 빼주세요', '물기를 뺀 파스타면을 그릇에 담아 토마토 마리네이드를 얹어 버무려주세요취향에 따라 파마산 치즈 등을 같이 곁들여 드시면 됩니다'], 'tips': []}","### 레시피 맛 설명\n이 레시피는 새콤달콤하고 시원한 맛이 특징입니다. 발사믹 식초와 레몬즙이 토마토의 신선한 맛을 강조하며, 꿀이 단맛을 더해줍니다. 올리브오일은 부드러운 질감을 제공하고, 파마산 치즈는 고소한 맛을 더해줍니다. 전체적으로 상큼하고 가벼운 맛이 특징인 요리입니다.\n\n### 핵심 재료와 비핵심 재료\n- **핵심 재료:**\n - **방울토마토:** 이 레시피의 주재료로, 마리네이드의 주된 맛을 형성합니다.\n - **발사믹식초, 레몬즙, 꿀:** 토마토의 맛을 새콤달콤하게 만들어주는 주요 조미료입니다.\n - **파스타면:** 요리의 주된 탄수화물 공급원으로, 마리네이드와 함께 먹기 위해 필요합니다.\n\n- **비핵심 재료:**\n - **양파, 파슬리가루:** 향과 맛을 더해주는 역할을 하지만, 다른 재료로 대체 가능성이 있습니다.\n - **파마산치즈:** 고소한 맛을 더하지만, 필수적이지는 않습니다.\n - **소금, 통후추:** 기본적인 간을 맞추기 위한 재료로, 다른 조미료로 대체 가능합니다.\n\n### 재료 변경 및 이유\n- **닭가슴살 추가:** 사용자가 반드시 사용해야 하는 재료로, 이 레시피에 단백질을 추가하여 영양가를 높일 수 있습니다. 닭가슴살은 파스타와 잘 어울리며, 마리네이드와 함께 조리하면 맛이 잘 배어들어 맛있게 즐길 수 있습니다.\n- **양파 대체:** 사용자가 양파를 싫어하거나 알레르기가 있는 경우, 브로콜리를 잘게 썰어 사용하면 식감과 영양을 더할 수 있습니다. 브로콜리는 마리네이드의 맛을 해치지 않으면서도 건강에 좋은 채소입니다.\n\n이러한 변경은 레시피의 기본 맛을 유지하면서도 사용자의 요구사항을 충족시킬 수 있습니다.","```\n[세부 분석]\n현재 레시피는 1인분 기준으로 작성되었습니다. 각 재료의 사용량을 1인분 기준으로 계산하여 1회 제공량을 기준으로 횟수를 구합니다.\n\n**채소류**에서 양파는 1개로 약 200g으로 1회 분량 70g 대비 약 2.857회입니다.\n**유지·당류**에 속하는 올리브오일은 10스푼으로 150g이며, 1회 분량 5g으로 나누면 30회입니다. 꿀은 1.5스푼으로 22.5g이며, 1회 분량 10g으로 나누면 2.25회입니다.\n**곡류**의 파스타면은 1컵으로 200g이며, 1회 분량 90g으로 나누면 2.222회입니다.\n**우유·유제품류**의 파마산치즈는 1스푼으로 15g이며, 이를 1회 분량 40g으로 나누면 0.375회입니다.\n방울토마토, 파슬리가루, 발사믹식초, 레몬즙, 소금, 통후추, 토마토 마리네이드는 분석 기준에서 분류되지 않으므로, **보조 재료**에 속하여 횟수 계산을 하지 않았습니다.\n\n이 계산은 각 재료의 사용량을 1인분 기준으로 변환한 후, 1회 제공량을 기준으로 분량과 횟수를 산출한 방식입니다.\n\n[요약]\n채소류: 2.857회분 (재료: 양파 200g)\n유지·당류: 32.25회분 (재료: 올리브오일 150g, 꿀 22.5g)\n곡류: 2.222회분 (재료: 파스타면 200g)\n우유·유제품류: 0.375회분 (재료: 파마산치즈 15g)\n보조: 방울토마토, 파슬리가루, 발사믹식초, 레몬즙, 소금, 통후추, 토마토 마리네이드는 식품군 기준에 해당하지 않는 재료이므로 횟수 계산에서 제외.\n```"
freq,4,1,1,1


In [102]:
# 데이터 가독성을 높여 출력하는 함수
def display_columns_with_format(df, col1, col2):
    for i, row in df.iterrows():
        print(f"Row {i + 1}:")
        print(f"--- {col1} ---")
        print(row[col1])  # 딕셔너리나 JSON 형태를 보기 좋게 출력
        print(f"\n--- {col2} ---")
        print(row[col2])  # 마크다운 텍스트 그대로 출력
        print("\n" + "=" * 40 + "\n")

# 함수 호출
display_columns_with_format(df, "recipe_info", "original_recipe_food_group_composition")


Row 1:
--- recipe_info ---
{'_id': '673c36d38fad4e3c931171ba', 'title': '토마토 마리네이드&냉파스타, 새콤달콤 시원한 별미', 'type_key': '샐러드', 'method_key': '절임', 'servings': '1인분', 'cooking_time': '60분 이내', 'difficulty': '초급', 'ingredients': ['방울토마토(1.5kg)', '양파(1개)', '파슬리가루(1스푼)', '올리브오일(10스푼)', '발사믹식초(5스푼)', '레몬즙(2스푼)', '꿀(1.5스푼)', '소금(약간)', '통후추(약간)', '토마토 마리네이드(1컵)', '파스타면(1컵)', '소금(1스푼)', '파마산치즈(1스푼)'], 'cooking_steps': ['양파는 잘게 다져 준비해주세요양파(소) 1개', '방울토마토는 끓는물에 약 2분간 데쳐주세요. 칼집을 내면 좀더 편하긴 한데 안해도 상관없어요방울토마토 1팩(약 1.5키로)중간불', '데친 후에는 물을 버리고 찬물에 담궈 식혀주고 토마토 껍질을 벗겨줍니다.칼집을 내지않아도 이렇게 껍질이 터져서 벗기기 쉽고 터져있지 않더라도 껍질 분리가 잘 된답니다^^', '껍질을 벗긴 토마토와 다진 양파, 파슬리가루를 넣어주세요step1의 다진양파, 파슬리가루 약 1스푼', '올리브오일, 레몬즙, 발사믹식초, 소금, 후추, 꿀을 요리재료에 적힌 분량대로 넣어준 뒤 잘 섞어주세요올리브오일 10스푼, 발사믹식초 5스푼, 레몬즙 2스푼 꿀 1.5스푼, 소금 약간, 통후추 약간', '완성된 토마토 마리네이드는 소독한 유리병에 담아 보관하고 먹을만큼 꺼내먹으면 됩니다^^', '소금 약 1스푼을 넣고 파스타를 심지가 씹히지않게 푹 삶아주세요소금 1스푼, 파스타(펜네) 1컵, 물 약500ml강불파스타면이나 푸실리, 마카로니 등 사용하셔도 됩니다.', '다 삶은 파스타면은 물기를 빼주세요', '물기를 뺀 파스타면을 그릇에 담아 토마토 마리네이드를 얹어 버무려주세요취향에

KeyError: 'original_recipe_food_group_composition'

In [ ]:
final_df.to_csv("data/test_dataset", index=False)